# Trámites TAD | Rutina de limpieza para publicación

In [1]:
import pandas as pd
import requests

In [66]:
FILENAME_INPUT = "input/TramitesTAD_Historico.csv"
FILENAME_OUTPUT = "output/tramites-tad-historico.csv"
FILENAME_OUTPUT_SERIES_NACION = "output/tramites-tad-series-nacion.csv"
FILENAME_OUTPUT_SERIES_PROVINCIAS = "output/tramites-tad-series-provincias.csv"

In [3]:
df = pd.read_csv(FILENAME_INPUT, encoding="utf8", sep=",")

## Chequeos / exploración

Última fecha de finalización

In [4]:
max(pd.to_datetime(df.FECHA_FINALIZACION))

Timestamp('2019-10-01 00:00:00')

Primera fecha de inicio

In [5]:
min(pd.to_datetime(df.FECHA_INICIO))

Timestamp('2016-07-01 00:00:00')

Cantidad de registros

In [6]:
len(df)

3262521

## Elimino columnas

In [7]:
df = df.drop("Unnamed: 0", axis=1, errors="ignore")
df = df.drop("FLAG_APODERADO", axis=1, errors="ignore")

## Agrega ids de provincias

In [8]:
sorted(df.PROVINCIA.unique())

['BUENOS AIRES',
 'CATAMARCA',
 'CHACO',
 'CHUBUT',
 'CIUDAD DE BUENOS AIRES',
 'CORDOBA',
 'CORRIENTES',
 'ENTRE RIOS',
 'FORMOSA',
 'JUJUY',
 'LA PAMPA',
 'LA RIOJA',
 'MENDOZA',
 'MISIONES',
 'NEUQUEN',
 'Otros',
 'RIO NEGRO',
 'SALTA',
 'SAN JUAN',
 'SAN LUIS',
 'SANTA CRUZ',
 'SANTA FE',
 'SANTIAGO DEL ESTERO',
 'Sin informar',
 'TIERRA DEL FUEGO',
 'TUCUMAN']

In [9]:
provincias_georef = pd.read_csv(
    "https://apis.datos.gob.ar/georef/api/provincias?aplanar&campos=id,nombre&orden=nombre&formato=csv", 
    dtype={"provincia_id": str}
)
provincias_georef

,provincia_id,provincia_nombre
0,06,Buenos Aires
1,10,Catamarca
2,22,Chaco
3,26,Chubut
4,02,Ciudad Autónoma de Buenos Aires
5,14,Córdoba
6,18,Corrientes
7,30,Entre Ríos
8,34,Formosa
9,38,Jujuy


In [10]:
PROVINCIAS_TAD_MAP = {
    'BUENOS AIRES': '06',
    'CATAMARCA': '10',
    'CHACO': '22',
    'CHUBUT': '26',
    'CIUDAD DE BUENOS AIRES': '02',
    'CORDOBA': '14',
    'CORRIENTES': '18',
    'ENTRE RIOS': '30',
    'FORMOSA': '34',
    'JUJUY': '38',
    'LA PAMPA': '42',
    'LA RIOJA': '46',
    'MENDOZA': '50',
    'MISIONES': '54',
    'NEUQUEN': '58',
    'RIO NEGRO': '62',
    'SALTA': '66',
    'SAN JUAN': '70',
    'SAN LUIS': '74',
    'SANTA CRUZ': '78',
    'SANTA FE': '82',
    'SANTIAGO DEL ESTERO': '86',
    'TIERRA DEL FUEGO': '94',
    'TUCUMAN': '90',
}

In [11]:
df["ID_PROVINCIA"] = df.PROVINCIA.map(PROVINCIAS_TAD_MAP)

## Agrega ids de localidades

In [12]:
API_BASE_URL = "https://apis.datos.gob.ar/georef/api/"

In [13]:
def get_similar_bulk(endpoint, nombres, **kwargs):
    """Normaliza una lista de nombres de alguna de las entidades geográficas."""

    assert len(nombres) <= 1000, "No se puede consultar más de 1000 a la vez"
    
    # construye lista de consultas
    data = {
        endpoint: [
            {**{"nombre": nombre, "max": 1}, **kwargs}
             for nombre in nombres
    ]}

    # realiza consulta bulk
    url = API_BASE_URL + endpoint
    results = requests.post(
        url, json=data, headers={"Content-Type": "application/json"}
    ).json()

    # convierte a una lista de "resultado más probable" o "vacío" cuando no hay
    parsed_results = [
        single_result[endpoint][0] if single_result[endpoint] else {}
        for single_result in results["resultados"]
    ]

    return parsed_results

In [14]:
pares_loc_prov = df[["LOCALIDAD", "ID_PROVINCIA"]].drop_duplicates()

In [15]:
pares_loc_prov.ID_PROVINCIA.value_counts()

06    884
14    445
82    323
50    194
30    178
90    153
54    119
66    111
18    105
70     97
22     94
86     93
62     89
26     78
58     73
38     72
74     68
42     68
10     64
46     54
34     54
02     49
78     26
94      8
Name: ID_PROVINCIA, dtype: int64

In [16]:
localidades_map = {}
for provincia_id in pares_loc_prov.ID_PROVINCIA.unique():
    localidades = list(pares_loc_prov[pares_loc_prov.ID_PROVINCIA == provincia_id].LOCALIDAD)
    
    if len(localidades) > 0:
        print("Buscando localidades de la provincia {}...".format(provincia_id))
        
        georef_results = get_similar_bulk("localidades", localidades, provincia=provincia_id, campos="id,nombre")
        for localidad, georef_result in zip(localidades, georef_results):
            localidades_map[(provincia_id, localidad)] = georef_result.get("id")

Buscando localidades de la provincia 02...
Buscando localidades de la provincia 06...
Buscando localidades de la provincia 14...
Buscando localidades de la provincia 50...
Buscando localidades de la provincia 82...
Buscando localidades de la provincia 86...
Buscando localidades de la provincia 90...
Buscando localidades de la provincia 30...
Buscando localidades de la provincia 70...
Buscando localidades de la provincia 66...
Buscando localidades de la provincia 26...
Buscando localidades de la provincia 38...
Buscando localidades de la provincia 58...
Buscando localidades de la provincia 78...
Buscando localidades de la provincia 74...
Buscando localidades de la provincia 62...
Buscando localidades de la provincia 94...
Buscando localidades de la provincia 22...
Buscando localidades de la provincia 46...
Buscando localidades de la provincia 42...
Buscando localidades de la provincia 18...
Buscando localidades de la provincia 54...
Buscando localidades de la provincia 10...
Buscando lo

In [17]:
df["ID_LOCALIDAD"] = df.set_index(['ID_PROVINCIA', 'LOCALIDAD']).index.map(localidades_map)

In [18]:
# chequeo de consistencia (deben ser todos True)
df_con_localidad_id = df[pd.notnull(df.ID_LOCALIDAD)]
(df_con_localidad_id.ID_LOCALIDAD.str[:2] == df_con_localidad_id.ID_PROVINCIA).value_counts()

True    3161058
dtype: int64

## Normalizo nombres de campos

In [19]:
df.columns

Index(['ID_TRAMITE', 'NOMBRE_TRAMITE', 'DESCRIPCION_CORTA', 'FECHA_INICIO',
       'FECHA_FINALIZACION', 'REPARTICION', 'ORGANISMO', 'ID_USUARIO',
       'TIPO_PERSONA', 'SEXO', 'LOGIN', 'LOCALIDAD', 'PROVINCIA',
       'ID_PROVINCIA', 'ID_LOCALIDAD'],
      dtype='object')

In [20]:
columns_new_names = {
    'ID_TRAMITE': 'tramite_id',
    'NOMBRE_TRAMITE': 'tramite_nombre',
    'DESCRIPCION_CORTA': 'tramite_descripcion',
    'FECHA_INICIO': 'fecha_inicio',
    'FECHA_FINALIZACION': 'fecha_finalizacion',
    'REPARTICION': 'reparticion_nombre',
    'ORGANISMO': 'organismo_nombre',
    'ID_USUARIO': 'usuario_id',
    'TIPO_PERSONA': 'persona_tipo',
    'SEXO': 'persona_sexo',
    'LOGIN': 'persona_login',
    'LOCALIDAD': 'localidad_nombre',
    'PROVINCIA': 'provincia_nombre',
    'ID_PROVINCIA': 'provincia_id',
    'ID_LOCALIDAD': 'localidad_id'
}

## Guarda archivo original procesado

In [21]:
df_renamed = df.rename(columns=columns_new_names)
df_renamed.to_csv(FILENAME_OUTPUT, encoding="utf8", sep=",", index=False)

## Produce distribuciones derivadas de series de tiempo

### Produce distribución de series a nivel nacional

In [40]:
# SERIES A NIVEL NACIONAL

## SERIES POR TRAMITES
## tramites-estado
serie_tad_tramites_iniciados = df_renamed.groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_finalizados = df_renamed.groupby("fecha_finalizacion").count()["tramite_id"]

## tramites-estado-persona_tipo
serie_tad_tramites_iniciados_pf = df_renamed[df_renamed.persona_tipo == "PF"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_pj = df_renamed[df_renamed.persona_tipo == "PJ"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_finalizados_pf = df_renamed[df_renamed.persona_tipo == "PF"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_pj = df_renamed[df_renamed.persona_tipo == "PJ"].groupby("fecha_finalizacion").count()["tramite_id"]

## tramites-estado-persona_sexo
serie_tad_tramites_iniciados_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].groupby("fecha_inicio").count()["tramite_id"]

serie_tad_tramites_finalizados_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].groupby("fecha_finalizacion").count()["tramite_id"]


## SERIES POR USUARIOS
## usuarios-estado
serie_tad_usuarios_inician = df_renamed.groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_finalizan = df_renamed.groupby("fecha_finalizacion")["usuario_id"].nunique()

## usuarios-estado-persona_tipo
serie_tad_usuarios_inician_pf = df_renamed[df_renamed.persona_tipo == "PF"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_pj = df_renamed[df_renamed.persona_tipo == "PJ"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pf = df_renamed[df_renamed.persona_tipo == "PF"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pj = df_renamed[df_renamed.persona_tipo == "PJ"].groupby("fecha_finalizacion")["usuario_id"].nunique()

## usuarios-estado-persona_sexo
serie_tad_usuarios_inician_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].groupby("fecha_inicio")["usuario_id"].nunique()

serie_tad_usuarios_finalizan_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].groupby("fecha_finalizacion")["usuario_id"].nunique()

In [68]:
SERIES_NACION = {
    "tad_tramites_iniciados": serie_tad_tramites_iniciados,
    "tad_tramites_finalizados": serie_tad_tramites_finalizados,
    "tad_tramites_iniciados_pf": serie_tad_tramites_iniciados_pf,
    "tad_tramites_iniciados_pj": serie_tad_tramites_iniciados_pj,
    "tad_tramites_finalizados_pf": serie_tad_tramites_finalizados_pf,
    "tad_tramites_finalizados_pj": serie_tad_tramites_finalizados_pj,
    "tad_tramites_iniciados_pf_sexo_masculino": serie_tad_tramites_iniciados_pf_sexo_masculino,
    "tad_tramites_iniciados_pf_sexo_femenino": serie_tad_tramites_iniciados_pf_sexo_femenino,
    "tad_tramites_iniciados_pf_sexo_no_declara": serie_tad_tramites_iniciados_pf_sexo_no_declara,
    "tad_tramites_finalizados_pf_sexo_masculino": serie_tad_tramites_finalizados_pf_sexo_masculino,
    "tad_tramites_finalizados_pf_sexo_femenino": serie_tad_tramites_finalizados_pf_sexo_femenino,
    "tad_tramites_finalizados_pf_sexo_no_declara": serie_tad_tramites_finalizados_pf_sexo_no_declara,
    "tad_usuarios_inician": serie_tad_usuarios_inician,
    "tad_usuarios_finalizan": serie_tad_usuarios_finalizan,
    "tad_usuarios_inician_pf": serie_tad_usuarios_inician_pf,
    "tad_usuarios_inician_pj": serie_tad_usuarios_inician_pj,
    "tad_usuarios_finalizan_pf": serie_tad_usuarios_finalizan_pf,
    "tad_usuarios_finalizan_pj": serie_tad_usuarios_finalizan_pj,
    "tad_usuarios_inician_pf_sexo_masculino": serie_tad_usuarios_inician_pf_sexo_masculino,
    "tad_usuarios_inician_pf_sexo_femenino": serie_tad_usuarios_inician_pf_sexo_femenino,
    "tad_usuarios_inician_pf_sexo_no_declara": serie_tad_usuarios_inician_pf_sexo_no_declara,
    "tad_usuarios_finalizan_pf_sexo_masculino": serie_tad_usuarios_finalizan_pf_sexo_masculino,
    "tad_usuarios_finalizan_pf_sexo_femenino": serie_tad_usuarios_finalizan_pf_sexo_femenino,
    "tad_usuarios_finalizan_pf_sexo_no_declara": serie_tad_usuarios_finalizan_pf_sexo_no_declara
}

In [84]:
df_nacion = pd.DataFrame(SERIES_NACION).reset_index().rename(columns={"index": "indice_tiempo"})

In [85]:
df_nacion.to_csv(FILENAME_OUTPUT_SERIES_NACION, index=False, encoding="utf8", sep=",")

In [87]:
# series a documentar en el catálogo
print(len(df_nacion.columns) - 1, "series")
for col in df_nacion.columns:
    print(col)

24 series
indice_tiempo
tad_tramites_iniciados
tad_tramites_finalizados
tad_tramites_iniciados_pf
tad_tramites_iniciados_pj
tad_tramites_finalizados_pf
tad_tramites_finalizados_pj
tad_tramites_iniciados_pf_sexo_masculino
tad_tramites_iniciados_pf_sexo_femenino
tad_tramites_iniciados_pf_sexo_no_declara
tad_tramites_finalizados_pf_sexo_masculino
tad_tramites_finalizados_pf_sexo_femenino
tad_tramites_finalizados_pf_sexo_no_declara
tad_usuarios_inician
tad_usuarios_finalizan
tad_usuarios_inician_pf
tad_usuarios_inician_pj
tad_usuarios_finalizan_pf
tad_usuarios_finalizan_pj
tad_usuarios_inician_pf_sexo_masculino
tad_usuarios_inician_pf_sexo_femenino
tad_usuarios_inician_pf_sexo_no_declara
tad_usuarios_finalizan_pf_sexo_masculino
tad_usuarios_finalizan_pf_sexo_femenino
tad_usuarios_finalizan_pf_sexo_no_declara


### Produce distribución de series a nivel provincial

In [52]:
# SERIES POR PROVINCIAS: se crean todas las series nacionales, con apertura provincial

## SERIES POR TRAMITES
## tramites-estado
df_prov_tad_tramites_iniciados = df_renamed.pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados = df_renamed.pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")

## tramites-estado-persona_tipo
df_prov_tad_tramites_iniciados_pf = df_renamed[df_renamed.persona_tipo == "PF"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_pj = df_renamed[df_renamed.persona_tipo == "PJ"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pf = df_renamed[df_renamed.persona_tipo == "PF"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pj = df_renamed[df_renamed.persona_tipo == "PJ"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")

## tramites-estado-persona_sexo
df_prov_tad_tramites_iniciados_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")

df_prov_tad_tramites_finalizados_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")


## SERIES POR USUARIOS
## usuarios-estado
df_prov_tad_usuarios_inician = df_renamed.pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan = df_renamed.pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

## usuarios-estado-persona_tipo
df_prov_tad_usuarios_inician_pf = df_renamed[df_renamed.persona_tipo == "PF"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_pj = df_renamed[df_renamed.persona_tipo == "PJ"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pf = df_renamed[df_renamed.persona_tipo == "PF"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pj = df_renamed[df_renamed.persona_tipo == "PJ"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

## usuarios-estado-persona_sexo
df_prov_tad_usuarios_inician_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

df_prov_tad_usuarios_finalizan_pf_sexo_masculino = df_renamed[df_renamed.persona_sexo == "M"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pf_sexo_femenino = df_renamed[df_renamed.persona_sexo == "F"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pf_sexo_no_declara = df_renamed[df_renamed.persona_sexo == "No declara"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

In [53]:
DFS_PROV = {
    "tad_tramites_iniciados": df_prov_tad_tramites_iniciados,
    "tad_tramites_finalizados": df_prov_tad_tramites_finalizados,
    "tad_tramites_iniciados_pf": df_prov_tad_tramites_iniciados_pf,
    "tad_tramites_iniciados_pj": df_prov_tad_tramites_iniciados_pj,
    "tad_tramites_finalizados_pf": df_prov_tad_tramites_finalizados_pf,
    "tad_tramites_finalizados_pj": df_prov_tad_tramites_finalizados_pj,
    "tad_tramites_iniciados_pf_sexo_masculino": df_prov_tad_tramites_iniciados_pf_sexo_masculino,
    "tad_tramites_iniciados_pf_sexo_femenino": df_prov_tad_tramites_iniciados_pf_sexo_femenino,
    "tad_tramites_iniciados_pf_sexo_no_declara": df_prov_tad_tramites_iniciados_pf_sexo_no_declara,
    "tad_tramites_finalizados_pf_sexo_masculino": df_prov_tad_tramites_finalizados_pf_sexo_masculino,
    "tad_tramites_finalizados_pf_sexo_femenino": df_prov_tad_tramites_finalizados_pf_sexo_femenino,
    "tad_tramites_finalizados_pf_sexo_no_declara": df_prov_tad_tramites_finalizados_pf_sexo_no_declara,
    "tad_usuarios_inician": df_prov_tad_usuarios_inician,
    "tad_usuarios_finalizan": df_prov_tad_usuarios_finalizan,
    "tad_usuarios_inician_pf": df_prov_tad_usuarios_inician_pf,
    "tad_usuarios_inician_pj": df_prov_tad_usuarios_inician_pj,
    "tad_usuarios_finalizan_pf": df_prov_tad_usuarios_finalizan_pf,
    "tad_usuarios_finalizan_pj": df_prov_tad_usuarios_finalizan_pj,
    "tad_usuarios_inician_pf_sexo_masculino": df_prov_tad_usuarios_inician_pf_sexo_masculino,
    "tad_usuarios_inician_pf_sexo_femenino": df_prov_tad_usuarios_inician_pf_sexo_femenino,
    "tad_usuarios_inician_pf_sexo_no_declara": df_prov_tad_usuarios_inician_pf_sexo_no_declara,
    "tad_usuarios_finalizan_pf_sexo_masculino": df_prov_tad_usuarios_finalizan_pf_sexo_masculino,
    "tad_usuarios_finalizan_pf_sexo_femenino": df_prov_tad_usuarios_finalizan_pf_sexo_femenino,
    "tad_usuarios_finalizan_pf_sexo_no_declara": df_prov_tad_usuarios_finalizan_pf_sexo_no_declara
}

In [54]:
def rename_columns_df_prov(df_prov, variable_name):
    df_prov.columns = df_prov.columns.map(lambda x: "{}_{}".format(variable_name, x))

In [56]:
for variable_name, df_prov in DFS_PROV.items():
    rename_columns_df_prov(df_prov, variable_name)

In [96]:
dfs_prov_merged = pd.concat([df_prov for df_prov in DFS_PROV.values()], axis=1).rename(columns={"fecha_inicio": "indice_tiempo"})
dfs_prov_merged = dfs_prov_merged.reset_index().rename(columns={"index": "indice_tiempo"})

In [97]:
dfs_prov_merged.to_csv(FILENAME_OUTPUT_SERIES_PROVINCIAS, index=False, encoding="utf8", sep=",")

In [98]:
# series a documentar en el catálogo
print(len(dfs_prov_merged.columns) - 1, "series")
for col in dfs_prov_merged.columns:
    print(col)

576 series
indice_tiempo
tad_tramites_iniciados_02
tad_tramites_iniciados_06
tad_tramites_iniciados_10
tad_tramites_iniciados_14
tad_tramites_iniciados_18
tad_tramites_iniciados_22
tad_tramites_iniciados_26
tad_tramites_iniciados_30
tad_tramites_iniciados_34
tad_tramites_iniciados_38
tad_tramites_iniciados_42
tad_tramites_iniciados_46
tad_tramites_iniciados_50
tad_tramites_iniciados_54
tad_tramites_iniciados_58
tad_tramites_iniciados_62
tad_tramites_iniciados_66
tad_tramites_iniciados_70
tad_tramites_iniciados_74
tad_tramites_iniciados_78
tad_tramites_iniciados_82
tad_tramites_iniciados_86
tad_tramites_iniciados_90
tad_tramites_iniciados_94
tad_tramites_finalizados_02
tad_tramites_finalizados_06
tad_tramites_finalizados_10
tad_tramites_finalizados_14
tad_tramites_finalizados_18
tad_tramites_finalizados_22
tad_tramites_finalizados_26
tad_tramites_finalizados_30
tad_tramites_finalizados_34
tad_tramites_finalizados_38
tad_tramites_finalizados_42
tad_tramites_finalizados_46
tad_tramites_fi